In [1]:
import numpy as np
import scipy.spatial as sp
import os
import csv
from datetime import datetime as time

In [2]:
#
# Running GO-ICP in a separate folder. In "temp" folder we put all the files on which the
# precompiled GO-ICP code is executed. The output is stored in "temp" and then collected by
# our go_icp function. Please do not alter the directory tree as it is important for proper
# functioning of the code below. All files in "temp" are wiped upon completion.  
#
def go_icp(A, B):
    #
    t = str(time.now()).replace(' ', '-')
    #
    # Data points: points of the source point set to be transformed.
    # Model points: points of the target point set.
    #
    filename_a   = 'go_icp/temp/data_{}.txt'.format(t)
    filename_b   = 'go_icp/temp/model_{}.txt'.format(t)
    filename_out = 'go_icp/temp/output_{}.csv'.format(t)
    #
    file = open(filename_a, 'w')
    print(' '.join([str(A.shape[1])]), file=file)
    for v in A.T:
        print(' '.join([str(x) for x in v]), file=file)
    file.close()
    #
    file = open(filename_b, 'w')
    print(' '.join([str(B.shape[1])]), file=file)
    for v in B.T:
        print(' '.join([str(x) for x in v]), file=file)
    file.close()
    #
    file = open(filename_out, 'w')
    print('NULL', file=file)
    file.close()
    #
    sample_size = min(A.shape[1], B.shape[1])
    #
    cmd = './go_icp/GoICP {} {} {} go_icp/config.txt {} >> go_icp/temp/echo.out'.format(filename_b, filename_a, sample_size, filename_out)
    os.system(cmd)
    #
    file = open(filename_out, 'r')
    reader = csv.reader(file)
    v = []
    for line in reader:
        v += [[float(x) for x in line]]
    file.close()
    #
    os.system('rm go_icp/temp/*')
    #
    v = np.array(v)
    #
    ortho = np.array(v[:,:-1])
    trans = np.array(v[:, -1])
    #
    return ortho, trans
    #

In [3]:
#
# Centering a point cloud A
#
def barycentered(A):
    #
    bar = np.sum(A, axis=1)/A.shape[1]
    #
    return A - bar[:, np.newaxis]

In [4]:
#
# Best orthogonal fit between two (centred) labelled point clouds
#
# A and B are assumed to have the same number of points
#
def best_fit(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na==nb
    #
    H = A @ B.T
    W, S, V = np.linalg.svd(H)
    #
    return V.T @ W.T

In [5]:
#
# Detecting nearest neighbours between two unlabelled point coluds A and B
#
# Assumption: cardinality of A <= cardinality of B 
#
# Input: point clouds A, B
#
# Output: for each point of A a matching point of B, as a 0/1-matrix
#
def nearest_neighbours(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na <= nb
    #
    tree = sp.KDTree(A.T, leafsize=10, compact_nodes=True, copy_data=True, balanced_tree=True)
    #
    matching = []
    I = matrix.identity(na)
    #
    for i in range(nb):
        _, ind = tree.query(B.T[i], k=1, p=2, workers=-1)
        matching += [I[ind]]
    #    
    return np.array(matching)

In [6]:
#
# ICP algorithm for two unlabelled point clouds A and B
#
# Assumption: cardinality of A <= cardinality of B
#
# Input:
# A, B = unlabelled point clouds
# init = initial orthogonal transformation
# max_iter = maximum number of iterations
# tol = tolerance for halting the computation
#
# Output:
# U = orthogonal transformation bringing A to B as close as possible
# nn = nearest neighbor matching between U*A and B
# dist = distance between U*A and B*nn (in the max singular value norm)
#
def icp(A, B, init=None, max_iter=100, tol=1e-16):
    #
    src = copy(A)
    dst = copy(B)
    #
    na = src.shape[1]
    nb = src.shape[1]
    #
    assert na <= nb
    #
    if (init is not None):
        src = init @ src
    #    
    prev_err = 0
    #
    for i in range(max_iter):
        #   
        nn = nearest_neighbours(src, dst)
        U = best_fit(src, dst @ nn)
        #    
        src = U @ src
        #
        diff = src - dst @ nn
        err  = np.linalg.norm(diff, 2)
        #
        if abs(prev_err - err) < tol:
            break
        #    
        prev_err = err
    #
    U = best_fit(A, src)
    nn = nearest_neighbours(U @ A, dst)
    #
    diff = U @ A - B @ nn
    dist = np.linalg.norm(diff, 2)
    #
    return U, nn, dist

In [7]:
#
# Testing ICP initialization on a given point cloud P
# 
# Producing another point cloud Q0 = O * P * S, with O a random orthogonal transformation
# and S a random permutation. Then adding noise N with each entry being Gaussian with
# mean 0 and standard deviation sigma. Creating Q = Q0 (Hadamard product) N.
#
# Test: determine the distance dist_spec from the recovered image of P denoted by 
# R = o * P * S to the specimen Q0, and compare it to the distance between d_icp 
# between o * P * s and Q returned by the ICP algorithm.
#
# Here it is important to note that S is a permutation, while s is a matching of nearest 
# neighbours in o * P and Q. The latter does not have to be a permutation matrix. 
#
def test_point_cloud_icp_init(P, sigma=0.05, tol=0.05, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    normP = np.linalg.norm(P, 2)
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q0 = O @ P @ S
    N = np.random.normal(1.0, sigma, (dim, num))
    Q = Q0 * N
    diff = Q - Q0
    dist_noise = np.linalg.norm(diff, 2) / normP
    if verbose:
        print("Noise introduced (normalised): {}".format(dist_noise))
    #
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    sols = []
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        nn = nearest_neighbours(U @ P, Q)
        diff_init = U @ P - Q @ nn
        d_init = np.linalg.norm(diff_init, 2) / normP
        sols += [(U, nn, d_init)]
    sols = sorted(sols, key=lambda x: x[2])
    o_init, s_init, dist_init = sols[0]
    o, s, d = icp(P, Q, o_init)
    #
    R = o @ P @ S
    diff = Q0 - R
    dist_spec = np.linalg.norm(diff, 2) / normP
    #
    diff = o - O
    dist_ortho = np.linalg.norm(diff, 2)
    #
    if verbose:
        print("Orthogonal transformation found:")
        print(o)
        print("Initial orthogonal transforamtion:")
        print(O)
        print("Distance to the initial one:")
        print(dist_ortho)
        print("Actual distance to specimen (normalised):")
        print(dist_spec)
        img  = point3d(Q0.T, size=2, color='blue')
        img += point3d(R.T,  size=2, color='red')
        img.plot().show()
    #
    flag = ( dist_spec < tol )
    #
    return flag, dist_noise, dist_spec, dist_ortho 

In [8]:
#
# Testing GO--ICP on a given point cloud P
# 
def test_point_cloud_go_icp(P, sigma=0.05, tol=0.05, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    normP = np.linalg.norm(P, 2)
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q0 = O @ P @ S
    N = np.random.normal(1.0, sigma, (dim, num))
    Q = Q0 * N
    diff = Q - Q0
    dist_noise = np.linalg.norm(diff, 2) / normP
    if verbose:
        print("Noise introduced (normalised): {}".format(dist_noise))
    #
    Q = barycentered(Q)
    #
    o, _ = go_icp(P, Q)
    o
    #
    diff = o - O
    dist_ortho = np.linalg.norm(diff, 2)
    #
    R = o @ P @ S
    diff = Q0 - R
    dist_spec = np.linalg.norm(diff, 2) / normP
    #
    if verbose:
        print("Orthogonal transformation found:")
        print(o)
        print("Initial orthogonal transforamtion:")
        print(O)
        print("Distance to the initial one:")
        print(dist_ortho)
        print("Actual distance to specimen (normalised):")
        print(dist_spec)
        img  = point3d(Q0.T, size=2, color='blue')
        img += point3d(R.T,  size=2, color='red')
        img.plot().show()
    #
    flag = ( dist_spec < tol )
    #
    return flag, dist_noise, dist_spec, dist_ortho

In [9]:
#
# ##################### Testing GO--ICP #####################
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
# The multiplicative noise used is Gaussian N(1, 0.1) in which case most tests pass. 
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #   
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    flag, dist_noise, dist_spec, dist_ortho\
    = test_point_cloud_go_icp(P, sigma=0.1, tol=0.05, verbose=True)
    #
    print("#"*42)

File read: Teapot.csv
Number of points: 351
Noise introduced (normalised): 0.09819115713603711
Orthogonal transformation found:
[[-0.801689 -0.531394 -0.273705]
 [ 0.539076 -0.444926 -0.715149]
 [ 0.258248 -0.720875  0.643153]]
Initial orthogonal transforamtion:
[[-0.79397305 -0.5442942  -0.2708332 ]
 [ 0.54579851 -0.44193845 -0.71189493]
 [ 0.26778868 -0.71304575  0.64796218]]
Distance to the initial one:
0.01540078050107957
Actual distance to specimen (normalised):
0.01443539541215292


Graphics3d Object

##########################################
File read: Bunny.csv
Number of points: 528
Noise introduced (normalised): 0.09609616405012261
Orthogonal transformation found:
[[-0.490654   0.838267   0.237839 ]
 [-0.292767   0.0984906 -0.951098 ]
 [-0.820699  -0.536291   0.197092 ]]
Initial orthogonal transforamtion:
[[-0.49106448  0.83992201  0.23105562]
 [-0.29097586  0.09185245 -0.95231097]
 [-0.82108996 -0.5348777   0.19929155]]
Distance to the initial one:
0.007233937400306728
Actual distance to specimen (normalised):
0.006790922969810803


Graphics3d Object

##########################################
File read: Cow.csv
Number of points: 602
Noise introduced (normalised): 0.07900881326805369
Orthogonal transformation found:
[[-0.768787  -0.0910717  0.632987 ]
 [ 0.285964  -0.934292   0.212892 ]
 [ 0.572006   0.34468    0.744315 ]]
Initial orthogonal transforamtion:
[[-0.76105728 -0.08412323  0.64320689]
 [ 0.28191307 -0.93591469  0.2111604 ]
 [ 0.58422328  0.34203359  0.73600012]]
Distance to the initial one:
0.015164589724572571
Actual distance to specimen (normalised):
0.014918961626841594


Graphics3d Object

##########################################


In [10]:
#
# ##################### Testing ICP initialization #####################
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
# The multiplicative noise used is Gaussian N(1, 0.1) in which case most tests pass. 
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #   
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    flag, dist_noise, dist_spec, dist_ortho\
    = test_point_cloud_icp_init(P, sigma=0.1, tol=0.05, verbose=True)
    #
    print("#"*42)

File read: Teapot.csv
Number of points: 351
Noise introduced (normalised): 0.08890461354858917
Orthogonal transformation found:
[[-0.56553472  0.80374186  0.18484992]
 [-0.81642749 -0.57731462  0.01240916]
 [ 0.11669033 -0.14389875  0.98268841]]
Initial orthogonal transforamtion:
[[-0.57079445  0.80049236  0.18277218]
 [-0.81284967 -0.58234957  0.01201661]
 [ 0.11605651 -0.14170729  0.98308185]]
Distance to the initial one:
0.0065652708766928405
Actual distance to specimen (normalised):
0.006450104221715127


Graphics3d Object

##########################################
File read: Bunny.csv
Number of points: 528
Noise introduced (normalised): 0.09437403918674102
Orthogonal transformation found:
[[-0.16072751 -0.66436264 -0.72992393]
 [ 0.84464661  0.29001765 -0.44995763]
 [ 0.51062587 -0.68884834  0.51453784]]
Initial orthogonal transforamtion:
[[-0.1604534  -0.66906836 -0.72567364]
 [ 0.84407735  0.28809438 -0.45225553]
 [ 0.51165236 -0.68509062  0.51851972]]
Distance to the initial one:
0.006348885818693388
Actual distance to specimen (normalised):
0.00540490193890186


Graphics3d Object

##########################################
File read: Cow.csv
Number of points: 602
Noise introduced (normalised): 0.08311488842543689
Orthogonal transformation found:
[[-0.72820782  0.16074027 -0.66624015]
 [-0.36833555 -0.91157145  0.18266474]
 [-0.57796392  0.37841782  0.72301982]]
Initial orthogonal transforamtion:
[[-0.72431627  0.15369463 -0.67211897]
 [-0.36921201 -0.90974651  0.18985198]
 [-0.58227866  0.38566728  0.7156901 ]]
Distance to the initial one:
0.011832100857333099
Actual distance to specimen (normalised):
0.005277748719985582


Graphics3d Object

##########################################


In [11]:
#
# ##################### Testing GO--ICP #####################
#
# Running a given number of tests num_tests on a given point cloud
# generated by calling test_point_cloud_go_icp(P, sigma, tol, verbose)
#
# Some statistics is collected and printed as output
#
def run_tests_point_cloud_go_icp(P, num_tests, sigma=0.05, tol=0.05, verbose=False):
    #
    num_success = 0
    num_fail  = 0
    rec_dist_noise = []
    rec_dist_spec = []
    rec_dist_ortho = []
    #
    start = time.now()
    #
    for i in range(num_tests):
        #
        msg = '### Test #{} : '.format(i+1)
        if verbose:
            print(msg)
        #
        test_flag, test_dist_noise,\
        test_dist_spec, test_dist_ortho = test_point_cloud_go_icp(P, sigma, tol, verbose)
        #
        rec_dist_noise += [test_dist_noise]
        rec_dist_spec += [test_dist_spec]
        rec_dist_ortho += [test_dist_ortho]
        #
        if test_flag:
            num_success += 1
        else:
            num_fail += 1
        #
        msg = msg + ' SUCCESS {}'.format(num_success)
        msg = msg + ' FAIL {}'.format(num_fail)
        print(msg, end='\r')
    #
    end = time.now()
    #
    assert (num_success + num_fail == num_tests)
    #
    avg_dist_noise = np.mean(rec_dist_noise)
    avg_dist_spec = np.mean(rec_dist_spec)
    avg_dist_ortho = np.mean(rec_dist_ortho)
    #
    delta_time = end-start
    print("Time elapsed (hours:minutes:seconds):", delta_time, ' '*42)
    avg_delta  = delta_time.total_seconds() / num_tests
    print("Average time per test (seconds):", avg_delta)
    print("Success rate:", float(num_success/num_tests))
    print("======= Mean values =======")
    print("Added noise (normalised): {}".format(avg_dist_noise))
    print("Distance to specimen (normalised): {}".format(avg_dist_spec))
    print("Distance to initial orthogonal transformation: {}".format(avg_dist_ortho))

In [12]:
#
# ##################### Testing ICP initialization #####################
#
# Running a given number of tests num_tests on a given point cloud
# generated by calling test_point_cloud_icp_init(P, sigma, tol, verbose)
#
# Some statistics is collected and printed as output
#
def run_tests_point_cloud_icp_init(P, num_tests, sigma=0.05, tol=0.05, verbose=False):
    #
    num_success = 0
    num_fail  = 0
    rec_dist_noise = []
    rec_dist_spec = []
    rec_dist_ortho = []
    #
    start = time.now()
    #
    for i in range(num_tests):
        #
        msg = '### Test #{} : '.format(i+1)
        if verbose:
            print(msg)
        #
        test_flag, test_dist_noise,\
        test_dist_spec, test_dist_ortho = test_point_cloud_icp_init(P, sigma, tol, verbose)
        #
        rec_dist_noise += [test_dist_noise]
        rec_dist_spec += [test_dist_spec]
        rec_dist_ortho += [test_dist_ortho]
        #
        if test_flag:
            num_success += 1
        else:
            num_fail += 1
        #
        msg = msg + ' SUCCESS {}'.format(num_success)
        msg = msg + ' FAIL {}'.format(num_fail)
        print(msg, end='\r')
    #
    end = time.now()
    #
    assert (num_success + num_fail == num_tests)
    #
    avg_dist_noise = np.mean(rec_dist_noise)
    avg_dist_spec = np.mean(rec_dist_spec)
    avg_dist_ortho = np.mean(rec_dist_ortho)
    #
    delta_time = end-start
    print("Time elapsed (hours:minutes:seconds):", delta_time, ' '*42)
    avg_delta  = delta_time.total_seconds() / num_tests
    print("Average time per test (seconds):", avg_delta)
    print("Success rate:", float(num_success/num_tests))
    print("======= Mean values =======")
    print("Added noise (normalised): {}".format(avg_dist_noise))
    print("Distance to specimen (normalised): {}".format(avg_dist_spec))
    print("Distance to initial orthogonal transformation: {}".format(avg_dist_ortho))

In [16]:
#
# ##################### Testing GO--ICP #####################
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
# The multiplicative noise used is Gaussian N(1, 0.1) in which case most tests pass. 
#
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #   
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    run_tests_point_cloud_go_icp(P, sigma=0.1, num_tests=10, tol=0.05, verbose=False)
    #
    print("#"*42)

File read: Teapot.csv
Time elapsed (hours:minutes:seconds): 0:12:39.006507                                           
Average time per test (seconds): 75.9006507
Success rate: 1.0
======= Mean values =======
Added noise (normalised): 0.09345925363150742
Distance to specimen (normalised): 0.00829218137570997
Distance to initial orthogonal transformation: 0.009480321524778405
##########################################
File read: Bunny.csv
Time elapsed (hours:minutes:seconds): 0:17:16.460320                                           
Average time per test (seconds): 103.64603199999999
Success rate: 1.0
======= Mean values =======
Added noise (normalised): 0.0879327298006458
Distance to specimen (normalised): 0.011298336597402772
Distance to initial orthogonal transformation: 0.012617589267439295
##########################################
File read: Cow.csv
Time elapsed (hours:minutes:seconds): 0:03:01.771613                                           
Average time per test (seconds): 18.17

In [14]:
#
# ##################### Testing ICP initialization #####################
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
#
# The multiplicative noise used is Gaussian N(1, 0.1) in which case most tests pass. 
#
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #   
    f.close()
    #    
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    run_tests_point_cloud_icp_init(P, sigma=0.1, num_tests=10, tol=0.05, verbose=False)
    #
    print("#"*42)

File read: Teapot.csv
Time elapsed (hours:minutes:seconds): 0:00:27.754140                                           
Average time per test (seconds): 2.775414
Success rate: 1.0
======= Mean values =======
Added noise (normalised): 0.08683062000834935
Distance to specimen (normalised): 0.006052111840189531
Distance to initial orthogonal transformation: 0.006962473363113212
##########################################
File read: Bunny.csv
Time elapsed (hours:minutes:seconds): 0:01:03.534574                                           
Average time per test (seconds): 6.3534574
Success rate: 1.0
======= Mean values =======
Added noise (normalised): 0.08870502319082312
Distance to specimen (normalised): 0.004993131162595841
Distance to initial orthogonal transformation: 0.005854649606932285
##########################################
File read: Cow.csv
Time elapsed (hours:minutes:seconds): 0:01:23.484963                                           
Average time per test (seconds): 8.348496299999